In [45]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import CMR_IA as cmr
sns.set_context('paper')
# pd.set_option('display.max_columns', None)

In [46]:
# pair recognition - cued recall
with open("../Data/simuS1_group2_design.pkl", 'rb') as inp:
    df_study = pickle.load(inp)
    df_test = pickle.load(inp)
# df_study = df_study.loc[df_study.session < 100]
# df_test = df_test.loc[df_test.session < 100]

In [47]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,session,pair_idx
0,1406,1272,SUN,SERVER,0,0
1,117,1287,BEGGAR,SHIRT,0,1
2,190,1261,BUILDER,SEAGULL,0,2
3,475,1439,DISHWASHER,TEA,0,3
4,374,94,COOKIE,BASEMENT,0,4
...,...,...,...,...,...,...
47995,1196,705,ROAD,HERD,999,43
47996,853,1122,MAGAZINE,PRIEST,999,44
47997,969,670,OBOE,GUITAR,999,45
47998,1393,1378,STREET,STEREO,999,46


In [48]:
df_test

,test_itemno1,test_itemno2,test_item1,test_item2,correct_ans,order,pair_idx,test,session
0,1109,898,PORPOISE,MINT,1,1,32,1,0
1,661,567,GRILL,FINGER,0,-1,-1,1,0
2,1086,732,PLATE,ICING,1,1,39,1,0
3,249,720,CARROT,HOSE,1,1,13,1,0
4,1385,340,STONE,COD,1,0,28,1,0
...,...,...,...,...,...,...,...,...,...
119995,165,-1,BRAIN,-1,983,1,8,2,999
119996,270,-1,CENT,-1,344,1,19,2,999
119997,1416,-1,SWAMP,-1,1289,1,4,2,999
119998,993,-1,OWNER,-1,1552,1,14,2,999


### Run CMR-IA

In [49]:
s_mat = np.load('../Data/wordpools/ltp_FR_similarity_matrix.npy')

In [50]:
params = cmr.make_default_params()
params.update(
    beta_enc = 0.5,
    beta_cue = 0.5,
    beta_rec = 0,
    beta_rec_new = 0.1, # beta for retrieved item
    beta_rec_post = 0.1,
    gamma_fc = 0.5,
    gamma_cf = 0.5,
    s_fc = 0.1,
    s_cf = 0.1,
    c_thresh = 0.01,
    c_thresh_ass = 0.70,
    a = 2800,
    b = 2.14,
    m = 0,
    n = 1,
    kappa = 0.02,
    lamb = 0,
    eta = 0,
    phi_s = 1,
    phi_d = 0.6,
    nitems_in_accumulator = 176,
    d_ass = 1,
    learn_while_retrieving = True,
    omega = 2,
    alpha = 1
)
params

{'beta_enc': 0.5,
 'beta_rec': 0,
 'beta_cue': 0.5,
 'beta_rec_new': 0.1,
 'beta_rec_post': 0.1,
 'beta_distract': None,
 'phi_s': 1,
 'phi_d': 0.6,
 's_cf': 0.1,
 's_fc': 0.1,
 'kappa': 0.02,
 'eta': 0,
 'omega': 2,
 'alpha': 1,
 'c_thresh': 0.01,
 'c_thresh_ass': 0.7,
 'd_ass': 1,
 'lamb': 0,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 176,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'a': 2800,
 'b': 2.14,
 'm': 0,
 'n': 1,
 'c1': 0,
 'No_recall': None,
 'gamma_fc': 0.5,
 'gamma_cf': 0.5}

In [51]:
df_simu, f_in_acc, f_in_dif = cmr.run_success_multi_sess(params,df_study, df_test ,s_mat)
df_simu

CMR2 Time: 117.02510094642639


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim
0,0,1109,898,1.0,589.666138,0.737952
1,0,661,567,0.0,2211.213623,0.120317
2,0,1086,732,1.0,575.782593,0.749086
3,0,249,720,1.0,608.018311,0.723631
4,0,1385,340,1.0,584.284668,0.742237
...,...,...,...,...,...,...
119995,999,165,-1,983.0,1380.000000,0.807277
119996,999,270,-1,344.0,1380.000000,0.786458
119997,999,1416,-1,1289.0,1430.000000,0.790738
119998,999,993,-1,1552.0,1360.000000,0.805601


In [52]:
df_simu = df_simu.merge(df_test,on=['session','test_itemno1','test_itemno2'])
df_simu['correct'] = df_simu.s_resp == df_simu.correct_ans
df_simu

,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_item1,test_item2,correct_ans,order,pair_idx,test,correct
0,0,1109,898,1.0,589.666138,0.737952,PORPOISE,MINT,1,1,32,1,True
1,0,661,567,0.0,2211.213623,0.120317,GRILL,FINGER,0,-1,-1,1,True
2,0,1086,732,1.0,575.782593,0.749086,PLATE,ICING,1,1,39,1,True
3,0,249,720,1.0,608.018311,0.723631,CARROT,HOSE,1,1,13,1,True
4,0,1385,340,1.0,584.284668,0.742237,STONE,COD,1,0,28,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,999,165,-1,983.0,1380.000000,0.807277,BRAIN,-1,983,1,8,2,True
119996,999,270,-1,344.0,1380.000000,0.786458,CENT,-1,344,1,19,2,True
119997,999,1416,-1,1289.0,1430.000000,0.790738,SWAMP,-1,1289,1,4,2,True
119998,999,993,-1,1552.0,1360.000000,0.805601,OWNER,-1,1552,1,14,2,True


In [53]:
# get f_in
sessions = np.unique(df_simu.session)
tmp_corr_fin = []
tmp_omax_fin = []
test1_num = 80

for sess in sessions:
    df_tmp = df_study.loc[df_study.session == sess]
    tmp1 = df_tmp.study_itemno1.to_numpy()
    tmp2 = df_tmp.study_itemno2.to_numpy()
    df_tmp2 = df_test.loc[df_test.session == sess]
    tmp3 = df_tmp2.test_itemno1[df_tmp2.test_itemno1 >= 0].to_numpy()
    tmp4 = df_tmp2.test_itemno2[df_tmp2.test_itemno2 >= 0].to_numpy()
    tmp = np.concatenate((tmp1, tmp2, tmp3, tmp4))
    tmp = np.unique(tmp)  # sort

    tmp_corr = df_simu.loc[df_simu.session == sess,"correct_ans"][test1_num:]
    corrid = np.searchsorted(tmp, tmp_corr)

    corr_fin = [f_in_dif[sess][i][id] for i, id in enumerate(corrid)]
    omax_fin = [np.max(np.delete(f_in_dif[sess][i], id)) for i, id in enumerate(corrid)]

    tmp_corr_fin = tmp_corr_fin + [-1] * test1_num + corr_fin
    tmp_omax_fin = tmp_omax_fin + [-1] * test1_num + omax_fin

df_simu['corr_fin'] = tmp_corr_fin
df_simu['omax_fin'] = tmp_omax_fin
df_simu

,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_item1,test_item2,correct_ans,order,pair_idx,test,correct,corr_fin,omax_fin
0,0,1109,898,1.0,589.666138,0.737952,PORPOISE,MINT,1,1,32,1,True,-1.000000,-1.000000
1,0,661,567,0.0,2211.213623,0.120317,GRILL,FINGER,0,-1,-1,1,True,-1.000000,-1.000000
2,0,1086,732,1.0,575.782593,0.749086,PLATE,ICING,1,1,39,1,True,-1.000000,-1.000000
3,0,249,720,1.0,608.018311,0.723631,CARROT,HOSE,1,1,13,1,True,-1.000000,-1.000000
4,0,1385,340,1.0,584.284668,0.742237,STONE,COD,1,0,28,1,True,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,999,165,-1,983.0,1380.000000,0.807277,BRAIN,-1,983,1,8,2,True,-0.262981,-0.532080
119996,999,270,-1,344.0,1380.000000,0.786458,CENT,-1,344,1,19,2,True,-0.264584,-0.537615
119997,999,1416,-1,1289.0,1430.000000,0.790738,SWAMP,-1,1289,1,4,2,True,-0.289134,-0.570070
119998,999,993,-1,1552.0,1360.000000,0.805601,OWNER,-1,1552,1,14,2,True,-0.254449,-0.594156


In [54]:
# recognition performance
df_recog = df_simu.query("test==1")
df_recog.groupby("correct_ans")["s_resp"].mean().to_frame(name="Yes rate")

,Yes rate
correct_ans,
0,0.000000
1,0.751525


In [55]:
# cued recall performance
df_cr = df_simu.query("test==2")
df_cr.correct.mean()

0.960625

In [56]:
def get_pair(df_tmp):
    df_tmp_pair = pd.pivot_table(df_tmp,index="pair_idx",columns="test",values="correct")
    df_tmp_pair.columns = ["test1","test2"]
    df_tmp_pair.reset_index(inplace=True)
    return df_tmp_pair

df_simu_p = df_simu.query("pair_idx >= 0")
df_pair = df_simu_p.groupby("session").apply(get_pair).reset_index()
df_pair

,session,level_1,pair_idx,test1,test2
0,0,0,0,0.0,1.0
1,0,1,1,0.0,1.0
2,0,2,2,0.0,1.0
3,0,3,3,1.0,1.0
4,0,4,4,0.0,1.0
...,...,...,...,...,...
39995,999,35,35,1.0,1.0
39996,999,36,36,1.0,1.0
39997,999,37,37,0.0,1.0
39998,999,38,38,1.0,1.0


In [57]:
df_tab = pd.crosstab(index=df_pair.test2,columns=df_pair.test1,normalize='all')
df_tab

test1,0.0,1.0
test2,,
0.0,0.010550,0.028825
1.0,0.237925,0.722700


In [58]:
def Yule_Q(A, B, C, D):
    return (A * D - B * C) / (A * D + B * C)
Yule_Q(df_tab[1][1],df_tab[0][1],df_tab[1][0],df_tab[0][0])

0.052911287052196034

In [59]:
# add 0.5 to each cell
Yule_Q(df_tab[1][1]+0.5,df_tab[0][1]+0.5,df_tab[1][0]+0.5,df_tab[0][0]+0.5)

0.23067549902467938